In [1]:
import torch.utils.data
import torch.nn as nn
import pickle

In [2]:
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 10

# Size of z latent vector (i.e. size of generator input)
latent_dim = 100

# Size of feature maps in generator
m_size = 100

# Number of training epochs
num_epochs = 50

# Learning rate for optimizers
lr = 0.001


In [3]:
#Loading dataset
with open('ds_train.pickle', 'rb') as f:
    dataset = pickle.load(f)

In [4]:

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

real_batch = next(iter(dataloader))

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(m_size,256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,m_size),
            nn.Sigmoid(),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [6]:
discriminator = Discriminator()

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(m_size, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 100),
        )

    def forward(self, x):
        output = self.model(x)
        return output

generator = Generator()

In [8]:
# Initialize BCELoss function
loss_function = nn.BCELoss()

In [9]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [10]:
for epoch in range(num_epochs):
    for n, (real_samples) in enumerate(dataloader,0):
        real_samples_labels = torch.ones((batch_size, 1))
        latent_space_samples = torch.randint(0, 2, (batch_size, 100, 100))
        latent_space_samples = latent_space_samples.float()
        generated_samples = generator(latent_space_samples)

        generated_samples_labels = torch.zeros((batch_size, 1))
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = loss_function(
            output_discriminator, all_samples_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Data for training the generator
        latent_space_samples = torch.randint(0, 2, (batch_size, 100, 100))
        latent_space_samples = latent_space_samples.float()

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples.detach()).view(-1)
        loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
        loss_generator.backward()
        optimizer_generator.step()

        # Show loss
        if epoch % 10 == 0 and n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")

ValueError: Using a target size (torch.Size([20, 1])) that is different to the input size (torch.Size([20, 100, 100])) is deprecated. Please ensure they have the same size.